
# POC 激活验证

In [2]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding, unpadding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP


# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    model(image)
    label = y_in
    break
del trainset, dataloader

In [3]:
# define the init process
def init_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size[0], kernel_size[1])
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    # print("kernel_size, stride, padding:")
    # print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    # print("conv input size:", data.size())
    x = operator(data.to("cuda"))

    return x


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['tag'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['kernel_params'][i]
        # do conv
        res_ft = init_conv(image, None, graph.ndata['kernel_weight'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        # do relu
        relu_opt = torch.nn.functional.relu
        res_ft = relu_opt(res_ft)

        # do max pooling
        pooling = graph.ndata['pooling_params'][i]
        if pooling.all() != 0:
            # do max_pooling
            kernel_size, stride, pad, dilation, ceil_mode = pooling
            max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                padding=pad, dilation=dilation, ceil_mode=ceil_mode)
            
            res_ft = max_pooling_operator(res_ft)
        _, _, r, c = res_ft.size()
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['ft_size'] = convd_size.to("cuda")



In [4]:
# init ft feature and get subg
with torch.no_grad():
    initiate_node_feature(g, image)
print(g.ndata['tag'])
# get node id
def nodes_with_feature_smaller_two(nodes):
    return nodes.data['tag'] <= 1
nodes_idx = g.filter_nodes(nodes_with_feature_smaller_two)
# print(nodes_idx)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None
# got the right subg of conv layers

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3],
       device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
       device='cuda:0')


In [13]:
from utils_gnn import equals, prepare_data
import pdb

def prepare_conv_data(nkernel_weight, 
                      nkernel_size, 
                      nkernel_bias, 
                      nkernel_params, 
                      channels):
    # prepare conv kernel weight
    _, width, height = nkernel_size
    kernel_weight = nkernel_weight
    kernel_weight = unpadding(kernel_weight, channels, width * height)
    kernel_weight = kernel_weight.reshape(1, channels, width, height)
    # prepare conv bias
    bias = nkernel_bias
    kernel_bias = unpadding(bias, 1, 1)[0]

    # prepare conv operator
    kernel_size, stride, pad = nkernel_params

    return kernel_weight, kernel_bias, kernel_size, stride, pad
    
def do_operation(data, conv_opt, pooling_params):
    # do conv
    torch.save(data, "./intermediate_data/data_test.pt")
    conv_ft = conv_opt(data.to("cuda")).to("cuda")
    _, _, width, height = conv_ft.size()
    np.savetxt("./intermediate_data/conv_ft.csv", conv_ft.reshape(width, height).cpu().numpy(), delimiter=',')
    # do relu
    relu_ft = torch.nn.functional.relu(conv_ft)
    np.savetxt("./intermediate_data/relu_ft.csv", relu_ft.reshape(width, height).cpu().numpy(), delimiter=',')
    ret_ft = relu_ft
    # do pooling
    if pooling_params.any() != 0:
        kernel_size, stride, pad, dilation, ceil_mode = pooling_params
        kernel_size, stride, pad, dilation, ceil_mode = int(kernel_size), int(stride), int(pad), int(dilation), bool(ceil_mode)
        max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
            padding=pad, dilation=dilation, ceil_mode=ceil_mode)
        
        ret_ft = max_pooling_operator(relu_ft)
        _, _, width, height = ret_ft.size()
        np.savetxt("./intermediate_data/ret_ft.csv", ret_ft.reshape(width, height).cpu().numpy(), delimiter=',')

    return ret_ft



def my_reduce(nodes):
    # received data and their size
    mfeat = nodes.mailbox['m']
    msize = nodes.mailbox['n']
    
    mf_size = mfeat.size()
    n_nodes = mf_size[0]
    n_channels = mf_size[1]

    # prepare ret_ft & ret_size, input size (1, 1, 28, 28)
    ret_ft = torch.zeros((n_nodes, 1, 1, 28, 28)).to("cuda")
    ret_size = torch.zeros((n_nodes, 2)).to("cuda")

    # for each node, do conv for received data
    for out_idx in range(n_nodes):

        # prepare received data and conv weight
        received_data = mfeat[out_idx]
        received_size = msize[out_idx]
        # skip if all zeros
        if equals(received_data, zeros=True):
            continue
        # (n, 1, 1, 28, 28) -> (1, n, width, height)
        data = prepare_data(received_data, received_size)

        # prepare conv operator data
        kernel_weight, kernel_bias, kernel_size, stride, pad = prepare_conv_data(nodes.data['kernel_weight'][out_idx], 
                                                                                nodes.data['kernel_size'][out_idx],
                                                                                nodes.data['bias'][out_idx], 
                                                                                nodes.data['kernel_params'][out_idx], 
                                                                                n_channels)
        
        # prepare conv operator
        conv_opt = torch.nn.Conv2d(n_channels, 1, kernel_size=kernel_size, stride=stride, padding=pad)
        conv_opt.weight.data = kernel_weight
        conv_opt.bias.data = kernel_bias

        # do operation
        ft = do_operation(data, conv_opt, nodes.data['pooling_params'][out_idx])

        # save ft size
        _, _, width, height = ft.size()
        np.savetxt("./intermediate_data/ret_ft.csv", ft.reshape(width, height).cpu().numpy(), delimiter=',')
        pad_ft = padding(ft.reshape(width, height), 28, 28)
        np.savetxt("./intermediate_data/pad_ft.csv", pad_ft.cpu().numpy(), delimiter=',')
        reshaped_ft = pad_ft.reshape(1, 1, 28, 28)
        # update return data
        ret_ft[out_idx] = reshaped_ft
        ret_size[out_idx] = torch.tensor([width, height]).to("cuda")
        
    # return size is [n, 1, 1, 28, 28], reduced from [n, m, 1, 1, 28, 28]
    return {'h': ret_ft, 'g': ret_size}

def my_message(edges):
    return {'m': edges.src['ft'], 'n': edges.src['ft_size']}



with torch.no_grad():
    subg.update_all(my_message, my_reduce)
    subg.ndata['ft'] += subg.ndata['h']
    subg.ndata['ft_size'] += subg.ndata['g']



In [6]:
print(subg.ndata['ft'].size())
print(subg.ndata['ft_size'].size())

torch.Size([88, 1, 1, 28, 28])
torch.Size([88, 2])


In [14]:
def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subg.nodes():
        id = int(id)
        feat = subg.nodes[id].data[key]
        if torch.eq(feat, torch.zeros(feat.size()).to("cuda")).all() == True:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)

print_zeros_in_feat('ft')
print_zeros_in_feat('h')
print_zeros_in_feat('ft_size')
print_zeros_in_feat('g')

ft: 
zeros -  [3, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
nonzeros -  [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
h: 
zeros -  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 46, 47, 49, 57, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
nonzeros -  [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63]
ft_size: 
zeros -  [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
nonzeros -  [0, 1

In [8]:
cnt = 0
ft_list = []
for f,s in zip(subg.ndata['ft'], subg.ndata['ft_size']):
    w, h = s
    w, h = int(w), int(h)
    data = unpadding(f.reshape(28, 28), w, h)
    ft_list.append(data)
    np.savetxt("./intermediate_data/graph_results/node-%d.csv" % cnt, data.cpu(), delimiter=',')
    cnt += 1

cnt = 0
cnn_params = None 
with open("./intermediate_data/params.json", "r") as f:
    cnn_params = json.load(f)
from torch.nn.functional import relu
from utils_gnn import unpadding
conv1 = torch.tensor(cnn_params['conv1'])
conv2 = torch.tensor(cnn_params['conv2'])
conv3 = torch.tensor(cnn_params['conv3'])
conv4 = torch.tensor(cnn_params['conv4'])
l = [conv1, conv2, conv3, conv4]
neural_list = []
for layer in l:
    for neural in layer[0]:
        neural_list.append(neural)
        np.savetxt("./intermediate_data/model_results/node-%d.csv" % cnt, neural.cpu(), delimiter=',')
        cnt += 1

In [10]:
print(conv1.size())

torch.Size([1, 32, 28, 28])


In [11]:
cnt = 0
f_cnt = -1
for f in ft_list[:32]:
    f_cnt += 1
    if f.all() == 0:
        print(f_cnt, ':', 0)
        continue
    d_cnt = -1
    for d in conv1[0]:
        d_cnt += 1
        try:
            print(torch.eq(h, d.to("cuda")).all())
            condition = torch.eq(h, d.to("cuda")).all() == True 
            if condition:
                print("equeals", f_cnt, d_cnt)
                cnt += 1
        except:
            continue
        

0 : 0
1 : 0
3 : 0
4 : 0
6 : 0
7 : 0
9 : 0
10 : 0
12 : 0
13 : 0
14 : 0
16 : 0
17 : 0
18 : 0
21 : 0
22 : 0
23 : 0
24 : 0
25 : 0
26 : 0
27 : 0
28 : 0
29 : 0
31 : 0
